In [43]:
import pandas as pd
import matplotlib.pyplot as plt
import re
import numpy as np
from nltk.stem import WordNetLemmatizer
import nltk
from nltk.corpus import stopwords
nltk.download('stopwords')
nltk.download('wordnet')
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
df = pd.read_csv('outputs/spicy.csv')
df.head()

[nltk_data] Downloading package stopwords to /home/rajiv/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /home/rajiv/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


,UserScreenName,UserName,Timestamp,Text,Embedded_text,Emojis,Comments,Likes,Retweets,Image link,Tweet URL
0,Melissa Foster,@Melissa_Foster,2021-03-21T11:13:30.000Z,The #NakedBaker is cooking up a spicy #hot #ro...,Seized by Love (The Ryders) - Melissa Foster A...,NaN,NaN,NaN,1.0,['https://pbs.twimg.com/card_img/1373244954315...,https://twitter.com/Melissa_Foster/status/1373...
1,CPK.Grownup,@Natasha78210065,2021-03-21T05:56:46.000Z,Check out #Hot and Spicy's video! #TikTok,#Hot and Spicy on TikTok\n#duet with @misscole...,😊 😁,NaN,NaN,NaN,['https://pbs.twimg.com/card_img/1373513895961...,https://twitter.com/Natasha78210065/status/137...
2,Nathan Blaser,@MorningStarGG,2021-03-21T18:26:15.000Z,Still #LIVE @ the #Asylum! #Playing #Overwatch...,MorningStarGG - Twitch\nLifelong gamer and dis...,NaN,NaN,NaN,1.0,['https://pbs.twimg.com/card_img/1373079231664...,https://twitter.com/MorningStarGG/status/13737...
3,CPK.Grownup,@Natasha78210065,2021-03-21T05:56:04.000Z,Check out #Hot and Spicy's video! #TikTok,#Hot and Spicy on TikTok\n#duet with @juicecup...,🤔 🙄 😬,NaN,NaN,NaN,['https://pbs.twimg.com/card_img/1373513724112...,https://twitter.com/Natasha78210065/status/137...
4,Vijeetha,@Vijeetha_NR,2021-03-22T08:43:47.000Z,I just don’t wanna look back & think “I could’...,NaN,NaN,NaN,NaN,2.0,['https://pbs.twimg.com/media/ExEjakOVkAISnV2?...,https://twitter.com/Vijeetha_NR/status/1373918...


In [44]:
#Define Wordcloud method
from wordcloud import WordCloud, STOPWORDS, ImageColorGenerator

def word_cloud(wd_list,filename):
    stopwords = set(STOPWORDS)
    all_words = ' '.join([text for text in wd_list])
    wordcloud = WordCloud(
        background_color='white',
        stopwords=stopwords,
        width=1600,
        height=800,
        random_state=1,
        colormap='jet',
        max_words=80,
        max_font_size=200).generate(all_words)
    plt.figure(figsize=(12, 10))
    plt.axis('off')
    plt.imshow(wordcloud, interpolation="bilinear");
    plt.savefig(f'../images/{filename}.png')
    plt.close()

In [45]:
def clean_data(text):
  #text = re.sub(r'', '', text) #Removes Html tag
  text = re.sub(r'[^\ a-zA-Z0-9]+', '', text)  #Removes non alphanumeric
  text = re.sub(r'^\s*|\s\s*', ' ', text).strip() #Removes extra whitespace, tabs
  stop_words = set(stopwords.words('english')) 
  lemmatizer = WordNetLemmatizer()
  text = text.lower().split() #Converts text to lowercase
  cleaned_text = list()
  for word in text:        
    if word in stop_words:    #Removes Stopwords, i.e words that don't convey any meaningful context/sentiments
      continue    
    word = lemmatizer.lemmatize(word, pos = 'v')    #Lemmatize words, pos = verbs, i.e playing, played becomes play
    cleaned_text.append(word)
  text = ' '.join(cleaned_text)
  return text

In [57]:
sent

'marvel lola'

In [49]:
from keras.models import model_from_json

# load json and create model
json_file = open('static/model/model.json', 'r')
loaded_model_json = json_file.read()
json_file.close()
loaded_model = model_from_json(loaded_model_json)
# load weights into new model
loaded_model.load_weights("static/model/model.h5")

In [72]:
import pickle
# loading
with open('tokenizer.pickle', 'rb') as handle:
    tokenizer = pickle.load(handle)

In [76]:
sent = "This is a bad morning!!!"
sent = clean_data(sent)
tokens = tokenizer.texts_to_sequences([sent])
tokens = pad_sequences(tokens, maxlen=280)
sentiment = loaded_model.predict(np.array(tokens), batch_size=1, verbose = 2)[0][0]
print('Sentiment =', sentiment)
if (round(sentiment) == 0):
    print('Negative')
else:
    print('Positive')

1/1 - 0s
Sentiment = 0.21663979
Negative
